In [ ]:
#@title Iniciar { vertical-output: true, display-mode: "both" }
from bs4 import BeautifulSoup
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import data_table
from datetime import *
import datetime as dt
import os
import ipywidgets as widgets
from IPython.display import display


def botao_salvar(r):
  button = widgets.Button(description="Salvar Excel")
  output = widgets.Output()
  def on_button_clicked(b):
    with output:
      r.salvarExcel()
  button.on_click(on_button_clicked)
  display(button, output)

class Relatorio:


  def __init__(self):
    self.funcionario=""
    self.empresa=""
    self.periodo=""

    self.arquivo = files.upload()
    self.nome_arquivo= self.__obterNomeArquivo()
    self.paginas = self.__carregarArquivo()
    self.dados_dic = self.__obterDados()
    self.dataFrame_geral = self.__criar_dataframe()

    self.diasNtrabalhados = 0
    self.totalDias = 0

    print("Data Frame pronto para uso")

  def __obterNomeArquivo(self):
    """ Obter Nome do Arquivo importado

    Returns:
    Nome do Primeirno Arquivo Dentro da pasta
    """
    dic_arquivo = dict(self.arquivo)
    print ('🟩 Arquivo Importado!')
    for n in dic_arquivo.keys():
      return str(n)
  
  def __carregarArquivo(self):
    try:
      print(self.nome_arquivo)
      html_file = open(self.nome_arquivo, "r", encoding="latin-1")
      html = html_file.read()
      soup = BeautifulSoup(html, 'html.parser')
      paginas = soup.find_all(class_='pagina')
      
      os.remove(self.nome_arquivo)
      print ('🟩 Lendo Arquivo!')
      return paginas
    except Exception as e:
      print(f'ERRO AO CARREGAR OS DADOS \n {e}')
      return ""

  def __preencher_campos_vazios(self, dados):
    dados['ENTRADA'].append("")
    dados['INTERVALO INICIO'].append("")
    dados['INTERVALO FINAL'].append("")
    dados['SAIDA'].append("")
    dados['LINHA'].append("0")
    dados['VEICULO'].append("0")
    dados['NORMAL'].append("")
    dados['ADICIONAL NOTURNO'].append("")
    dados['HORA EXTRA'].append("")
    dados['INCOM'].append("")
    dados['TOTAL'].append("")
    return dados

  def __tratar_ocorrencias(self, l, dados):
    if l[12:27] == '** F O L G A **':
      dados['OCORRENCIA'].append('FOLGA')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)
    
    if l[12:27] == '-- F A L T A --':
      dados['OCORRENCIA'].append('FALTA')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)

    if l[12:27] == 'LICENCA REMUNER':
      dados['OCORRENCIA'].append('LICENCA REMUNERADA')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)

    if l[12:19] == 'FERIADO':
      dados['OCORRENCIA'].append('FERIADO')
      self.__preencher_campos_vazios(dados)
      return (False, dados)

    if l[12:23] == 'PATERNIDADE':
      dados['OCORRENCIA'].append('PATERNIDADE')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)

    if l[12:17] == 'GREVE':
      dados['OCORRENCIA'].append('GREVE')
      self.__preencher_campos_vazios(dados)
      return (False, dados)

    if l[12:27] == 'FOLGA DECRET ES':
      dados['OCORRENCIA'].append('FOLGA DECRETO ES')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)
      
    if l[12:24] == 'ATESTADO MED':
      dados['OCORRENCIA'].append('ATESTADO MEDICO')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)

    if l[12:45] == '                     ** FERIAS **':
      dados['OCORRENCIA'].append('FERIAS')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)
    
    if l[26:33] == 'Data de':
      dados['OCORRENCIA'].append('DEMISSAO')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)
    
    if l[12:21] == 'SUSPENSAO':
      dados['OCORRENCIA'].append('SUSPENSAO')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)
    
    if l[12:17] == 'OBITO':
      dados['OCORRENCIA'].append('OBITO')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)
    
    if l[12:25] == 'LIC N/REMUNER':
      dados['OCORRENCIA'].append('LICENCA NAO REMUNERADA')
      dados = self.__preencher_campos_vazios(dados)
      return (False, dados)

    return (True, dados)

  def __verificar_nulo(self, l):
    if l == "":
      return "0"
    return l

  def __remover_asterisco(self, t):
    return t.replace('*', '')

  def __tratar_tempo(self, t):
    if len(t) == 1:
      t = ""
      return t
    return t

  def __tratar_linha_reserva(self, t):
    if t == "RESE":
      return '999'
    return t


  def __tratarDados(self, dados):
    try:
      dados['LINHA'] = list(map(self.__tratar_linha_reserva, dados['LINHA'] ))
      dados['VEICULO'] = list(map(self.__remover_asterisco, dados['VEICULO'] ))

      dados['HORA EXTRA'] = list(map(self.__tratar_tempo, dados['HORA EXTRA'] ))
      dados['ENTRADA'] = list(map(self.__tratar_tempo, dados['ENTRADA'] ))
      dados['INTERVALO INICIO'] = list(map(self.__tratar_tempo, dados['INTERVALO INICIO'] ))
      dados['INTERVALO FINAL'] = list(map(self.__tratar_tempo, dados['INTERVALO FINAL'] ))
      dados['SAIDA'] = list(map(self.__tratar_tempo, dados['SAIDA'] ))
      dados['NORMAL'] = list(map(self.__tratar_tempo, dados['NORMAL'] ))
      dados['ADICIONAL NOTURNO'] = list(map(self.__tratar_tempo, dados['ADICIONAL NOTURNO'] ))
      dados['INCOM'] = list(map(self.__tratar_tempo, dados['INCOM'] ))
      dados['TOTAL'] = list(map(self.__tratar_tempo, dados['TOTAL'] ))
      return dados
    except Exception as e:
      print(f'ERRO AO TRATAR OS DADOS \n {e}')
      return ""


  def __obterDados(self):
    print ('🟩 Processando Dados!')
    dados_dic = {'DATA': [], 'ENTRADA': [], 'INTERVALO INICIO': [], 'INTERVALO FINAL': [], 'SAIDA': [], 'LINHA': [], 'VEICULO':[], 
          'OCORRENCIA': [], 'NORMAL':[], 'ADICIONAL NOTURNO':[], 'HORA EXTRA': [], 'INCOM': [], 'TOTAL': []}
    
    for i in range(len(self.paginas)):
      
      linha = self.paginas[i]
      linha = str(linha).replace('<div class="pagina">', '')
      for cont in range(1000):
        linha = linha.replace(f'<div style="top:{str(cont)}px;left:0px;">', '')
      linha = linha.replace('<div style="top:112px;left:566px;Width:45px;text-align:right;">', '')
      lista = linha.split('</div>')

      self.empresa = lista[3][14:36]
      self.funcionario = lista[10][24:53]
      self.periodo = lista[1][83:107]
      
      for count in range(15):
        del lista[0]
      
      del lista[len(lista)-1]
      del lista[len(lista)-1]
      del lista[len(lista)-1]

      for l in lista:
        dados_dic['DATA'].append(l[:8].replace(" ", ""))
        ocorrencia_tratada = self.__tratar_ocorrencias(l, dados_dic)
        dados_dic = ocorrencia_tratada[1]

        if ocorrencia_tratada[0]:
          dados_dic['ENTRADA'].append(self.__verificar_nulo(l[11:16].replace(" ", "")))
          dados_dic['INTERVALO INICIO'].append(self.__verificar_nulo(l[17:22].replace(" ", "")))
          dados_dic['INTERVALO FINAL'].append(self.__verificar_nulo(l[23:28].replace(" ", "")))
          dados_dic['SAIDA'].append(self.__verificar_nulo(l[29:34].replace(" ", "")))
          dados_dic['LINHA'].append(self.__verificar_nulo(l[41:45].replace(" ", "")))
          dados_dic['VEICULO'].append(self.__verificar_nulo(l[52:59].replace(" ", "")))
          dados_dic['OCORRENCIA'].append(self.__verificar_nulo(l[60:67].replace(" ", "")))
          dados_dic['NORMAL'].append(self.__verificar_nulo(l[76:83].replace(" ", "")))
          dados_dic['ADICIONAL NOTURNO'].append(self.__verificar_nulo(l[83:90].replace(" ", "")))
          dados_dic['HORA EXTRA'].append(self.__verificar_nulo(l[90:97].replace(" ", "")))
          dados_dic['INCOM'].append(self.__verificar_nulo(l[124:130].replace(" ", "")))
          dados_dic['TOTAL'].append(self.__verificar_nulo(l[132:137].replace(" ", "")))

          dados_dic = self.__tratarDados(dados_dic)

    return dados_dic
  

  def __criar_dataframe(self):
    try:
      print ('🟩 Criando Tabela')
      df = pd.DataFrame(data=self.dados_dic)
      df = df[df.VEICULO != "-------"]
      df = df[df.DATA != ""]
      df = df[df.DATA != "DATA"]  
      print ('🟩 Ajustando Tabela')
      df.VEICULO = df.VEICULO.astype(int)
      return df
    except Exception as e:
      print(f'ERRO AO CRIAR DATA FRAME \n {e}')
      return ""
  
  def salvarExcel(self):
    self.dataFrame_geral.to_csv('Relatorio.csv', encoding='UTF-8', index=False, sep=';')
    if files.download('Relatorio.csv'):
      os.remove('Relatorio.csv')

  def contagem(self, campo):
    ndf = pd.DataFrame(data=self.dataFrame_geral[campo].value_counts(sort=True))
    ndf.rename(columns={campo: 'QUANTIDADE'}, inplace=True)
    ndf[campo] = ndf.index
    ndf = ndf[[campo, 'QUANTIDADE']]
    ndf.reset_index(drop=True, inplace=True)
    return ndf

  def __EscreverLinhaHTML(self, a, b):
    linha = "                           <tr>\n                              <th scope='row'>" +a+ "</th>\n                              <td>"+b+"</td>\n                            </tr>\n"
    return  linha
  
  def __escreverTabela(self, pagina, dataframe, campo, quantidadeLinhas):

    tamanho= 0

    if (quantidadeLinhas != "full"):
      if len(dataframe.index) >=10 :
        tamanho = 10
    else:
      tamanho = len(dataframe.index)
    
    for l in range(tamanho):
      a = str(dataframe.iloc[l][str(campo)])
      b = str(dataframe.iloc[l]['QUANTIDADE'])
      pagina.write(self.__EscreverLinhaHTML(a,b))
  
  def gerarHTML(self):
    ocorrencias = self.contagem(str("OCORRENCIA"))
    linhas = self.contagem(str("LINHA"))
    veiculos = self.contagem(str("VEICULO"))

    self.totalDias = ocorrencias['QUANTIDADE'].sum()

    for l in range(len(ocorrencias.index)):
      if ocorrencias.iloc[l]["OCORRENCIA"] == "DTRAB":
        self.diasNtrabalhados = self.totalDias - int(ocorrencias.iloc[l]["QUANTIDADE"])
  
    for l in range(len(linhas.index)):
      if linhas.iloc[l]["LINHA"] == "0":
        linhas.loc[linhas.LINHA=="0", 'QUANTIDADE'] = int(linhas.iloc[l]["QUANTIDADE"] - self.diasNtrabalhados)
        linhas = linhas.sort_values(by=['QUANTIDADE'], ascending=False  )

    for l in range(len(veiculos.index)):
      print(veiculos.iloc[l]["VEICULO"])
      if veiculos.iloc[l]["VEICULO"] == 0: 
        print('foi')
        veiculos.loc[veiculos.VEICULO==0, 'QUANTIDADE'] = int(veiculos.iloc[l]["QUANTIDADE"] - self.diasNtrabalhados)
        veiculos = veiculos.sort_values(by=['QUANTIDADE'], ascending=False  )
    
    

    pagina = open("index.html", "w")
    print('Arquivo criado')
      

    pagina.write("<!DOCTYPE html> \n<html lang='pt-BR'>\n"  
    + "<head>\n <meta charset='utf-8'>\n  <link href='https://cdn.jsdelivr.net/npm/bootstrap@5.2.0/dist/css/bootstrap.min.css' rel='stylesheet' />  \n <link href='https://fonts.googleapis.com/css2?family=Ubuntu:ital,wght@0,300;0,400;0,500;0,700;1,300;1,400;1,500;1,700&display=swap' rel='stylesheet'>\n" 
    + " <style> .campo{ margin-left: 9px; font-size: 18px;  margin-top: 4px; }\n"
    + " .label{ margin-left: 5px;  font-size: 20px;  font-weight: bold; }\n"
    + " .titulo-tabela{ background-color: #020a16; color: #fff; padding: 10px;  text-shadow: 1px 1px 1px rgba(176, 180, 180, 0.822);  margin-bottom: 0;  font-size: 14px; }\n"
    + " .grafico {  width: 100vh; }\n"
    + " tr:hover {  box-shadow: 1px 1px 1px rgba(10, 12, 12, 0.822); }\n" 
    + " .informacoes{  margin-top: 20px;  margin-left: 2px;  margin-bottom: 18px;   background-color: #e6e6e6;     padding: 5px;}\n" 
    + " .boxinfo{ height: 60px; padding: 5px;  font-weight: 700; font-size: 12px; text-align: center; }\n"
    + "</style>\n"  
    + "<title>Relatório</title>\n</head>\n" 
    + "<body>\n "
    + "<header>\n" 
    + "<script type='text/javascript' src='https://www.gstatic.com/charts/loader.js'></script>\n"
    + "<script type='text/javascript'>\n"
    + "google.charts.load('current', {'packages':['corechart']});\n"
    + "google.charts.setOnLoadCallback(drawChart);\n"
    + "function drawChart() {\n"
    + "var data = google.visualization.arrayToDataTable([\n"
    + "['Ocorrencia', 'Quantidade', { role: 'annotation' }  ],\n")
    
    for i in range(len(ocorrencias.index)):
      pagina.write("[ '"+ ocorrencias.iloc[i]['OCORRENCIA']+"', "+str(ocorrencias.iloc[i]['QUANTIDADE'])+", '"+ str(ocorrencias.iloc[i]['QUANTIDADE'])+"' ],\n")
    

    pagina.write("]);\n"
    + "var view = new google.visualization.DataView(data);\n"
    + "view.setColumns([0, 1,\n"
    + "{ calc: 'stringify',\n"
    + "sourceColumn: 1,\n"
		+ "type: 'string',\n"
    + "role: 'annotation' },\n"
    + "2]);\n var options = {\n title: 'Gráfico de Ocorrências',\n width: 700,\n"
    + "height: 300, \n is3D: true, \n bar: {groupWidth: '70%'}, \n legend: { position: 'none' },\n };"
    + "var chart = new google.visualization.ColumnChart(document.getElementById('chart_div'));\n"
    + "chart.draw(data, options);\n } \n </script>"
    + ""
		
    + "<div class='container'>\n"
    + "<div class='col border-bottom'>\n"
    + "<div class='row'>\n"
    + "<h1>Relatório Geral de Atividade</h1>\n"
    + "</div>\n"
    + "<div class='row'>\n" \
    + "<p class='lead'>PERÍODO DE " +self.periodo+"</p>\n" 
    + "</div>\n"
    + "</div>\n"
    + "<div class='row informacoes'>\n" 
    + "<div class='col-4'>\n" 
    + "<div class='row'>\n" 
    + "<p class='label'>Empresa:</p><p class='campo'>"+ self.empresa +"</p>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "<div class='col'>\n" 
    + "<div class='row'>\n" 
    + "<p class='label'>Funcionário:</p><p class='campo'>"+ self.funcionario +"</p>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</header>\n\n" 
    + "<main>\n" 
    + "<div class='container'>\n" 
    + "<div class='row'>\n" 
    + "<div class='col tabela'>\n" 
    + "<h5 class='titulo-tabela'>OCORRÊNCIAS</h5>\n" 
    + "<table class='table table-striped'>\n" 
    + "<thead class='thead-light'>\n" 
    + "<th scope='col'>TIPO</th>\n" 
    + "<th scope='col'>QUANTIDADE</th>\n" 
    + "</thead>\n" 
    + "<tbody class='corpo-tabela'>\n")

    #montar tabela de Ocorrência
    self.__escreverTabela(pagina, ocorrencias, 'OCORRENCIA', "full")

    pagina.write("</tbody>\n" 
    + "</table>\n" 
    + "</div>\n" 
    + "<div class='col align-self-center'>\n" 
    + "<div id='chart_div'></div>\n" 
    + "</div>\n" 
    + "</div>\n"
    + "<hr>\n" 
    + "<div class='row'>\n" 
    + "<div class='col'>\n" 
    + "<h5 class= 'titulo-tabela'>10 LINHAS MAIS TRABALHADAS</h5>\n" 
    + "<table class='table table-striped'>\n" 
    + "<thead class='thead-light'>\n" 
    + "<th scope='col'>LINHA</th>\n" 
    + "<th scope='col'>QUANTIDADE</th>\n" 
    + "</thead>\n" 
    + "<tbody>\n")

    #Montar Tabela de Linha
    self.__escreverTabela(pagina, linhas, 'LINHA', "limitado")

    pagina.write("</tbody>\n" 
    + "</table>\n" 
    + "</div>\n" 
    + "<div class='col'>\n" 
    + "<h5 class= 'titulo-tabela'>10 VEÍCULOS MAIS TRABALHADOS</h5>\n" 
    + "<table class='table table-striped'>\n" 
    + "<thead class='thead-light'>\n" 
    + "<th scope='col'>VEÍCULO</th>\n" 
    + "<th scope='col'>QUANTIDADE</th>\n" 
    + "</thead>\n" 
    + "<tbody>\n")

    #Montar Tabela de Veículo
    self.__escreverTabela(pagina, veiculos, 'VEICULO',  "limitado")

    pagina.write("</tbody>\n" 
    + "</table>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</main>\n" 
    + "<footer>\n" 
    + "<hr>\n" 
    + "<div class='container'>\n" 
    + "<div class='row'>\n" 
    + "<div class='col'>\n" 
    + "<h6>"+self.empresa+"</h6>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</div>\n" 
    + "</footer>\n" 
    + "<script src='https://code.jquery.com/jquery-3.5.1.min.js'></script>\n" 
    + "<script src='https://cdn.jsdelivr.net/npm/bootstrap@5.2.0/dist/js/bootstrap.bundle.min.js'</script>\n" 
    + "</body>\n" 
    + "\n" 
    + "</html>\n")

    pagina.close()

    print("ARQUIVO INDEX.HTML GERADO!")
    if files.download('index.html') :
      os.remove('index.html')


print("Tudo Pronto")

Tudo Pronto


In [ ]:
#@title Importar Arquivo
novo = Relatorio()

Saving -------------------.HTML to -------------------.HTML
🟩 Arquivo Importado!
-------------------.HTML
🟩 Lendo Arquivo!
🟩 Processando Dados!
🟩 Criando Tabela
🟩 Ajustando Tabela
Data Frame pronto para uso


In [ ]:
#@title Visualizar Dados

botao_salvar(novo)

novo.dataFrame_geral

Button(description='Salvar Excel', style=ButtonStyle())

Output()

,DATA,ENTRADA,INTERVALO INICIO,INTERVALO FINAL,SAIDA,LINHA,VEICULO,OCORRENCIA,NORMAL,ADICIONAL NOTURNO,HORA EXTRA,INCOM,TOTAL
2,08/04/17,,,,,0,0,FOLGA,,,,,
3,09/04/17,07:35,10:45,11:25,15:30,0605,12175,DTRAB,07:15,,,00:05,07:15
4,10/04/17,04:45,09:00,10:00,13:10,0605,12175,DTRAB,07:20,00:17,00:05,,07:25
5,11/04/17,04:45,09:00,10:00,13:10,0605,12175,DTRAB,07:20,00:17,00:05,,07:25
6,12/04/17,04:45,09:00,10:00,13:10,0605,12175,DTRAB,07:20,00:17,00:05,,07:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,11/06/20,04:30,10:30,11:30,12:45,4052,0,DTRAB,07:15,00:34,,00:05,07:15
1215,12/06/20,04:35,10:30,11:30,11:35,0612,12140,DTRAB,06:00,00:29,,01:20,06:00
1216,13/06/20,,,,,0,0,FOLGA,,,,,
1217,14/06/20,04:30,10:10,11:10,14:00,0658,12168,DTRAB,07:20,00:34,01:10,,08:30


In [ ]:
#@title Relatório de Linha, Veículo e Ocorrência  { run: "auto", vertical-output: true, display-mode: "form" }
#@markdown Selecione o Relatório Desejado.
option = "LINHA" #@param ['LINHA', 'VEICULO', 'OCORRENCIA' ]

data_table.disable_dataframe_formatter()

df = novo.contagem(str(option))

botao_salvar(df)


df

Button(description='Salvar Excel', style=ButtonStyle())

Output()

,LINHA,QUANTIDADE
0,0605,664
1,0,333
2,0612,32
3,0651,28
4,1000,11
5,4052,10
6,0658,6
7,0616,6
8,0604,6
9,0617,5


In [ ]:
novo.gerarHTML()